In [ ]:
# A program to analyze scanlag images

In [ ]:
!rm -rf Scanlag/
! git clone https://github.com/baliga-lab/Scanlag.git

Cloning into 'Scanlag'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 31 (delta 15), reused 4 (delta 0), pack-reused 0
Unpacking objects: 100% (31/31), 23.70 KiB | 1.48 MiB/s, done.


In [ ]:
from Scanlag import scanlag_analyzer as sa

In [ ]:
# Import folders from google drive
# The data should be in the users account. If not, upload the data.

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# getting project info for data paths

project_dir = 'msm-llr-scanlag2-09062022'

In [ ]:
# Folders with scanlag images

analysis_dir = '/content/drive/MyDrive/baliga-lab/AMR/scanlag-analysis/' + project_dir + '/'
scanned_images = '/content/drive/MyDrive/baliga-lab/AMR/scanlag-data/images/' + project_dir + '/'

In [ ]:
# Initite analysis

output_dir = sa.inittiate_analysis(analysis_dir,scanned_images)

In [ ]:
# Identify plates

plate_pos = sa.find_plates(output_dir,scanned_images)

*   Examine the annotatted image. Plates that are not identified should be added manually. This can be done using ImageJ program. plate locations should be added as -
  `plate_pos['n']: [x,y,550]` ; where,
  n: plate number, x&y: x,y co-ordinates of the center of plates found with ImageJ.
*   Name the plates by examining the annotated scan image



In [ ]:
# Name plates
# Do not use space, pound sign (#) or underscores (_) in names

# SL1
# plate_names = {'1':'B.I.pre.minus',
#                '2':'D.I.pre.minus',
#                '3':'F.I.pre.minus',
#                '4':'A.I.pre.minus',
#                '5':'C.I.pre.minus',
#                '6':'E.I.pre.minus'}
# plate_names = {'1':'B.R.plus',
#                '2':'C.NA.minus',
#                '3':'A.NA.minus',
#                '4':'C.R.plus',
#                '5':'A.R.plus',
#                '6':'B.NA.minus'}

# SL2
# plate_names = {'1':'H.I.pre.minus',
#                '2':'D.I.post.minus',
#                '3':'B.I.post.minus',
#                '4':'G.I.pre.minus',
#                '5':'C.I.post.minus',
#                '6':'A.I.post.minus'}
plate_names = {'1':'B.RI.plus',
               '2':'A.I.plus',
               '3':'C.I.plus',
               '4':'C.RI.plus',
               '5':'B.I.plus',
               '6':'A.RI.plus'}

# SL3
# plate_names = {'1':'H.I.post.minus',
#                '2':'F.I.post.minus',
#                '3':'B.I.post.plus',
#                '4':'E.I.post.minus',
#                '5':'G.I.post.minus',
#                '6':'A.I.post.plus'}

# SL4
# plate_names = {'1':'D.I.post.plus',
#                '2':'F.I.post.plus',
#                '3':'H.I.post.plus',
#                '4':'E.I.post.plus',
#                '5':'C.I.post.plus',
#                '6':'G.R.t15.plus'}

In [ ]:
# Crop plates

cropped_plates, max_times = sa.crop_plates(scanned_images,output_dir,plate_pos,plate_names)

In [ ]:
# Set size threshold

cropped_plates = '/content/drive/MyDrive/baliga-lab/AMR/scanlag-analysis/' + project_dir + '/cropped_images'
#max_times = [3997]*6 # this is t-3 for SL1
max_times = [4026]*6 # this is t-3 for SL2
#max_times = [4184]*6 # this is t-3 for SL3
#max_times = [4190]*6 # this is t-3 for SL4

# this is how you can change the max time to be specific for each plate
# max_times = [2160, 1000, etc.]

In [ ]:
# Define parameters

size = [25, 3000] #This is the range of colony sizes that is allowed.\
                  #Decrease upper bound if doublets are identified as colonies.\
                  #Increase lower bound if anamolies such as bubbles and shadows are identified as CFUs.

plates_to_analyze = [i for i in plate_names.values()] #This can be changed to skip plates, see below
#plates_to_analyze = [i for i in plate_names.values() if i not in ['C.R.t15.plus', 'D.R.t15.plus', 'E.R.t15.plus', 'F.R.t15.plus']]

times_to_annotate = max_times #Time list are the times from which images are used to identify CFUs, \
                             #maximum times are used as a default \
                             #it can be reduced if the CFUs are overgrown and are overlapping each other.


growth = sa.get_growth(cropped_plates,plates_to_analyze,times_to_annotate,size,output_dir)
# this was done to test
#growth = sa.get_growth(cropped_plates,plate_names,times_to_annotate,size,output_dir)
growth_normalized_to_spots = sa.normalize_to_spots(growth)

In [ ]:
growth_normalized_to_spots.to_csv('/content/drive/MyDrive/baliga-lab/AMR/scanlag-data/msm-output-data-09062022/' + project_dir + '-growth.csv')
#time_of_appearance.to_csv('/content/drive/MyDrive/baliga-lab/AMR/scanlag-data/msm-output-data-02172023/' + project_dir + '-toa.csv')

Inspect time vs growth plot and identify the time to use for TOA identification

In [ ]:
# Calculate time of appearance
# 3500 seems to be a good threshold value for the Msm LLR plates
#pixel_threshold = 3500
pixel_threshold = 3500

time_of_appearance = sa.get_time_of_appearance(growth_normalized_to_spots,pixel_threshold)

In [ ]:
#print(growth_normalized_to_spots)
print(time_of_appearance.sort_values(by = 'TOA').head(n = 60))

                    TOA     Cell_type      Spot_loc
Spot                                               
B.R.t0.minus_4    738.0  B.R.t0.minus     25_714_33
C.R.t0.minus_3    738.0  C.R.t0.minus     49_228_53
A.R.t0.minus_37   738.0  A.R.t0.minus  1014_1020_34
A.R.t0.minus_38   738.0  A.R.t0.minus   1039_932_49
C.R.t0.minus_16   738.0  C.R.t0.minus   338_1036_41
A.R.t0.minus_8    738.0  A.R.t0.minus    173_801_35
B.R.t0.minus_29   738.0  B.R.t0.minus   482_1006_38
B.R.t0.minus_60   738.0  B.R.t0.minus    966_847_48
A.R.t0.minus_35   738.0  A.R.t0.minus   966_1070_34
D.R.t0.minus_0    738.0  D.R.t0.minus     23_233_38
B.R.t0.minus_11   738.0  B.R.t0.minus    112_511_42
B.R.t0.minus_12   738.0  B.R.t0.minus   121_1056_48
C.R.t0.minus_52   738.0  C.R.t0.minus   1055_495_35
E.R.t0.minus_32   738.0  E.R.t0.minus    992_641_44
A.R.t0.minus_11   738.0  A.R.t0.minus   216_1033_30
C.R.t0.minus_50   738.0  C.R.t0.minus   1012_432_49
E.R.t0.minus_34   738.0  E.R.t0.minus   998_1027_32
B.R.t0.minus

In [ ]:
# Plot spot intensity vs time
#sa.plot_time_vs_growth(growth_normalized_to_spots,"Growth_normalized",Cell_type)

In [ ]:
# Plot TOA

import seaborn as sns
from matplotlib import pyplot as plt

In [ ]:
def mask_image(image,size):
    labels = measure.label(image)
    mask = np.zeros(image.shape,dtype="uint8")
    for label in np.unique(labels):
        if label == 0:
            continue
        labelMask = np.zeros(image.shape,dtype="uint8")
        labelMask[labels == label]=255
        numPixels = cv2.countNonZero(labelMask)
        if numPixels > size[0] and numPixels < size[1]:
            mask = cv2.add(mask,labelMask)
    #cv2.imwrite("Test/Masked_image.png",mask)
    return mask

In [ ]:
from imutils import contours
help(contours.sort_contours)

Help on function sort_contours in module imutils.contours:

sort_contours(cnts, method='left-to-right')



In [ ]:
print(time_of_appearance)

             TOA Cell_type     Spot_loc
Spot                                   
3-a-0_0    966.0     3-a-0   30_1047_42
3-a-0_1    736.0     3-a-0    24_661_35
3-a-0_10   828.0     3-a-0   345_145_36
3-a-0_11   736.0     3-a-0  371_1087_46
3-a-0_12   828.0     3-a-0    587_84_27
...          ...       ...          ...
5-b-0_0   1196.0     5-b-0    12_165_15
5-b-0_1   1150.0     5-b-0   206_235_22
5-b-0_2   1150.0     5-b-0  251_1087_19
5-b-0_3   1288.0     5-b-0   473_470_23
5-b-0_4   1242.0     5-b-0  717_1083_19

[209 rows x 3 columns]
